## Mission 10 Stored procedures using python


Once, the connection is established, and you have created a cursor object, select the database `little_lemon` for use. 

In [5]:
# Set the little_lemon database for use 
cursor.execute("use little_lemon")

# Confirm the database in use
connection.database

'little_lemon'

## Exercise: 

Little lemon needs to perform some tasks on daily basis, and they involve extracting data from one or more tables. The tasks include finding the guest with maximum spending, retrieving the bookings for no arrival, and displaying the order status to the guests. To keep consistency during the data retrieval process, Little Lemon is interested to implement the required tasks using stored procedures.

#### Task 1:

Little lemon is running a marketing campaign this month. They need to issue a discount coupon to the top spender on daily basis. Create a stored procedure `TopSpender` that can retrieve the booking ID, guest’s full name, and the bill amount of the top spender at closing. Call the procedure and print the results.   

In [7]:
cursor.execute("DROP PROCEDURE IF EXISTS NoArrival;")

In [6]:
cursor.execute("DROP PROCEDURE IF EXISTS TopSpender;")
find_top_spender = """create procedure TopSpender()
begin 
select Bookings.BookingID, concat( Bookings.GuestFirstName, ' ', Bookings.GuestLastName), Orders.BillAmount 
from Bookings inner join Orders on Bookings.BookingID = Orders.BookingID order BY BillAmount DESC LIMIT 1;
end """
cursor.execute(find_top_spender)
cursor.callproc("TopSpender")
results = next(cursor.stored_results())
dataset = results.fetchall()
for data in dataset:
    print(data)

(1, 'Anna Iversen', 86)


#### Task 2:

Help Little Lemon to retrieve all those bookings where the guests did not appear today.  Use `NoArrival` as a name for your stored procedure.  

In [54]:
cursor.execute("DROP PROCEDURE IF EXISTS NoArrival;")

In [55]:
no_show = """create procedure NoArrival() 
begin
select Bookings.BookingID, concat(Bookings.GuestFirstName, ' ', Bookings.GuestLastName) as "GuestName", Orders.BillAmount 
from Bookings left join Orders on Bookings.BookingID = Orders.BookingID where BillAmount is null ;
end """
cursor.execute(no_show)
cursor.callproc("NoArrival")
results = next (cursor.stored_results())
dataset = results.fetchall()
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]
print(columns)
for data in dataset:
    print(data)

['BookingID', 'GuestName', 'BillAmount']
(6, 'Diana Pinto', None)


#### Task 3:

It is very important for little lemon to keep track of the status of each guest’s order and display it on the screen to keep their guests informed.  

This is how they categorize the orders: 

* If not assigned to any employee, the status is `In Queue` 
* If assigned to the employees with IDs 4 and 5, the status is `Preparing Order` 
* If assigned to the employees with IDs 1, 2, and 3, the status is `Order Served` 

Create a stored procedure named `OrderStatus` for the above task and call to check if everything is working.  

In [20]:
cursor.execute("DROP PROCEDURE IF EXISTS OrderStatus;")
see_orders = """ create procedure OrderStatus() 
begin
select BookingID, case
when EmployeeID in (1,2,3) then "Order Served"
when EmployeeID in (4,5) then "Preparing Order"
else "In Queue"
end as "StatusToday" from Bookings;
end"""

cursor.execute(see_orders)
cursor.callproc("OrderStatus")
results = next(cursor.stored_results())
dataset = results.fetchall()
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

print(columns)
for data in dataset:
    print (data)


['BookingID', 'StatusToday']
(1, 'Order Served')
(2, 'Order Served')
(3, 'Order Served')
(4, 'Preparing Order')
(5, 'Order Served')
(6, 'Preparing Order')


In [21]:
if connection.is_connected():
    cursor.close()
    print("The cursor is closed.")
    connection.close()
    print("MySQL connection is closed.")
else:
    print("Connection is already closed")

The cursor is closed.
MySQL connection is closed.
